Wrangling Data with MongoDB kE


In [ ]:
from pprint import PrettyPrinter

import pandas as pd
from IPython.display import VimeoVideo
from pymongo import MongoClient

In [ ]:
#Instantiate a PrettyPrinter, and assign it to the variable pp
pp = PrettyPrinter(indent=2)

Prepare Data

Connect

In [ ]:
#Create a client that connects to the database running at localhost on port 27017
client = MongoClient(host='localhost', port=27017)

Explore

In [ ]:
client.list_databases()

In [ ]:
list(client.list_databases())

In [ ]:
#Print a list of the databases available on client
pp.pprint(list(client.list_databases())

In [ ]:
#Assign the "air-quality" database to the variable db
db = client['air-quality']

In [ ]:
#Use the list_collections method to print a list of the collections available in db
list(db.list_collections())

In [ ]:
for c in db.list_collections():
    print(c["name"])

In [ ]:
#Assign the "nairobi" collection in db to the variable name nairobi
nairobi = db['nairobi']

In [ ]:
#Use the count_documents method to see how many documents are in the nairobi collection
nairobi.count_documents({})

In [ ]:
#Use the find_one method to retrieve one document from the nairobi collection, and assign it to the variable name result
result = nairobi.find_one({})
pp.pprint(result)

In [ ]:
#Use the distinct method to determine how many sensor sites are included in the nairobi collection.
nairobi.distinct("metadata.site")

In [ ]:
#Use the count_documents method to determine how many readings there are for each site in the nairobi collection
print("Documents from site 6:", nairobi.count_documents({"metadata.site":6}) )
print("Documents from site 29:", nairobi.count_documents({"metadata.site":29}))

In [ ]:
#Use the aggregate method to determine how many readings there are for each site in the nairobi collection
result = nairobi.aggregate(
    [
        {"$group":{"_id": "$metadata.site", "count":{"$count": {}}}}
    ]
)
pp.pprint(list(result))

In [ ]:
# Use the distinct method to determine how many types of measurements have been taken in the nairobi collection
nairobi.distinct("metadata.measurement")

In [ ]:
#Use the find method to retrieve the PM 2.5 readings from all sites. Be sure to limit your results to 3 records only
result = nairobi.find({"metadata.measurement":"P2"}).limit(3)
pp.pprint(list(result))

In [ ]:
#Use the aggregate method to calculate how many readings there are for each type ("humidity", "temperature", "P2", and "P1") in site 6
result=nairobi.aggregate(
    [
        {"$match": {"metadata.site":6}},
    {"$group": {"_id": "$metadata.measurement", "count":{"$count":{}}}}
]
)
pp.pprint(list(result))

In [ ]:
#Use the aggregate method to calculate how many readings there are for each type ("humidity", "temperature", "P2", and "P1") in site 29
result=nairobi.aggregate(
    [
        {"$match": {"metadata.site":29}},
    {"$group": {"_id": "$metadata.measurement", "count":{"$count":{}}}}
]
)
pp.pprint(list(result))

Import


In [ ]:
#Use the find method to retrieve the PM 2.5 readings from site 29. Be sure to limit your results to 3 records only. Since we won't need the metadata for our model, use the projection argument to limit the results to the "P2" and "timestamp" keys only
result = nairobi.find(
    {"metadata.site":29, "metadata.measurement":"P2"},
    projection= { "P2":1, "timestamp":1, "_id":0}
)
pp.pprint(result.next())

In [ ]:
#Read records from your result into the DataFrame df. Be sure to set the index to "timestamp"
df = pd.DataFrame(result).set_index("timestamp")
df.head()